### **Problem Statement**
The problem is to predict the probability of heart disease based on patient data using a **Bayesian Network**.

### **Inputs:**
- **Dataset:** Cleveland Heart Disease dataset (`04_heart.csv` from Kaggle)
- dataset: https://www.kaggle.com/datasets/ritwikb3/heart-disease-cleveland
- **Features Used:**
  - `age` (Age of the patient)
  - `sex` (Gender: 1 = Male, 0 = Female)
  - `cp` (Chest pain type)
  - `trestbps` (Resting blood pressure)
  - `chol` (Cholesterol level)
  - `fbs` (Fasting blood sugar)
  - `restecg` (Resting ECG results)
  - `thalach` (Maximum heart rate achieved)
  - `exang` (Exercise-induced angina)
  - `oldpeak` (ST depression)
  - `slope` (Slope of peak exercise ST segment)
  - `ca` (Number of major vessels colored by fluoroscopy)
  - `thal` (Thalassemia test result)
  - `heartdisease` (Target variable: 0 = No Disease, 1-4 = Different levels of disease)

### **Outputs:**
- **Bayesian Network Model** trained using Maximum Likelihood Estimation (MLE).
- **Probabilistic Inference** using **Variable Elimination** to compute:
  1. Probability of `heartdisease` given `age=35`
  2. Probability of `heartdisease` given `chol=205`


This Bayesian model helps in **diagnosing heart disease risk** based on different health parameters.

In [1]:
import numpy as np
import pandas as pd

* `numpy` is used for numerical computations.
* `pandas` is used for handling and processing data in tabular format (like CSV files).

In [2]:

from pgmpy.models import BayesianModel

* `BayesianModel` is used to define a Bayesian Network, a probabilistic graphical model.
* It represents dependencies between different variables using `Directed Acyclic Graphs` (DAGs).

In [3]:
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination

* `MaximumLikelihoodEstimator` is used to estimate the parameters (probabilities) of the Bayesian Network using Maximum Likelihood Estimation (MLE).
* MLE finds the parameter values that maximize the likelihood of the observed data.


* `VariableElimination` is an algorithm for probabilistic inference in Bayesian Networks.
* It helps answer queries about the model, such as:
    * "What is the probability of disease given symptoms?"
    * "What is the most likely cause of an event?"

In [4]:
# Read Cleveland Heart Disease data
heartDisease = pd.read_csv("04_heart.csv")
heartDisease.replace("?", np.nan, inplace=True)  # Replace '?' with NaN

In [12]:
# Display the first few rows of the dataset
print("Few examples from the dataset:")
heartDisease.head() # print first 5 rows

Few examples from the dataset:


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,heartdisease
0,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0


In [6]:
# Define Bayesian Network structure
Model = BayesianModel(
    [
        ("age", "trestbps"),
        ("age", "fbs"),
        ("sex", "trestbps"),
        ("exang", "trestbps"),
        ("trestbps", "heartdisease"),
        ("fbs", "heartdisease"),
        ("heartdisease", "restecg"),
        ("heartdisease", "thalach"),
        ("heartdisease", "chol"),
    ]
)

![imga](imgs\e4_bayesian_network.png)

In [ ]:
# Learning CPDs using Maximum Likelihood Estimators
print("\nLearning CPD using Maximum Likelihood Estimators")

Model.fit(heartDisease, estimator=MaximumLikelihoodEstimator)


Learning CPD using Maximum Likelihood Estimators


In [8]:
# Inference with Bayesian Network
HeartDisease_infer = VariableElimination(Model)

In [9]:
print("Unique ages:", heartDisease["age"].unique())
print("Unique chol values:", heartDisease["chol"].unique())

Unique ages: [63 67 37 41 56 62 57 53 44 52 48 54 49 64 58 60 50 66 43 40 69 59 42 55
 61 65 71 51 46 45 39 68 47 34 35 29 70 77 38 74 76]
Unique chol values: [233 286 229 250 204 236 268 354 254 203 192 294 256 263 199 168 239 275
 266 211 283 284 224 206 219 340 226 247 167 230 335 234 177 276 353 243
 225 302 212 330 175 417 197 198 290 253 172 273 213 305 216 304 188 282
 185 232 326 231 269 267 248 360 258 308 245 270 208 264 321 274 325 235
 257 164 141 252 255 201 222 260 182 303 265 309 307 249 186 341 183 407
 217 288 220 209 227 261 174 281 221 205 240 289 318 298 564 246 322 299
 300 293 277 214 207 223 160 394 184 315 409 244 195 196 126 313 259 200
 262 215 228 193 271 210 327 149 295 306 178 237 218 242 319 166 180 311
 278 342 169 187 157 176 241 131]


In [10]:
# Computing the Probability of HeartDisease given Age=35
print("\n1. Probability of HeartDisease given Age=35")
q1 = HeartDisease_infer.query(variables=["heartdisease"], evidence={"age": 35})
print(q1)


1. Probability of HeartDisease given Age=35
+-----------------+---------------------+
| heartdisease    |   phi(heartdisease) |
+=================+=====================+
| heartdisease(0) |              0.7109 |
+-----------------+---------------------+
| heartdisease(1) |              0.0945 |
+-----------------+---------------------+
| heartdisease(2) |              0.0233 |
+-----------------+---------------------+
| heartdisease(3) |              0.1395 |
+-----------------+---------------------+
| heartdisease(4) |              0.0319 |
+-----------------+---------------------+


In [11]:
# Computing the Probability of HeartDisease given Cholesterol=100
print("\n2. Probability of HeartDisease given Cholesterol=205")
q2 = HeartDisease_infer.query(variables=["heartdisease"], evidence={"chol": 205})
print(q2)


2. Probability of HeartDisease given Cholesterol=205
+-----------------+---------------------+
| heartdisease    |   phi(heartdisease) |
+=================+=====================+
| heartdisease(0) |              0.4926 |
+-----------------+---------------------+
| heartdisease(1) |              0.0000 |
+-----------------+---------------------+
| heartdisease(2) |              0.0000 |
+-----------------+---------------------+
| heartdisease(3) |              0.5074 |
+-----------------+---------------------+
| heartdisease(4) |              0.0000 |
+-----------------+---------------------+


### **Final Predictions:**
1. **For Age = 35**  
   - Highest probability: **No Heart Disease (0.7109 or 71.09%)**
   - Lower probabilities for higher heart disease levels.

2. **For Cholesterol = 205**  
   - `heartdisease(3)` (Severe heart disease) has the highest probability: **0.5074 (50.74%)**
   - No chance for `heartdisease(1,2,4)`, moderate chance for `heartdisease(0)`